# Setup

First, let us do some setup. 

* Create a SageMaker execution role. This role should have access to S3 and permission to create SageMaker HPO jobs. Save the ARN of this role. We need to paste this ARN in the line of code that defines `role` in our Lambda function later. 

* Create a SQS queue, note the URL of your queue and set `queue_url` in Lambda Function to this URL later. 

# Lambda 

### Create Lambda layer for SageMaker Python SDK

`sagemakersdk.zip` is provided in this repo. Run the following code to create `sagemakersdk` layer. Make sure your enter the right bucket name in the script below. 

Please note `sagemakersdk` layer must work together with `AWSLambda-Python37-SciPy1x` layer. `AWSLambda-Python37-SciPy1x` is provided by AWS and you don't need to create it yourself. We will add both layers to our Lambda function in later step. 

In [ ]:
%%sh
aws s3 cp sagemakersdk.zip <s3 bucket>
aws lambda publish-layer-version --layer-name sagemakersdk --content S3Bucket=<s3 bucket>,S3Key=sagemakersdk.zip --compatible-runtimes python3.7

### Create Lambda function

The code for Lambda function looks like below. It checks SQS queue for messages first. Each message contains hyperparameter ranges in message body. The Lambda function creates HPO jobs only if HPO job limit is not reached.

In [ ]:
# %load hpo_lambda.py
import json
import sagemaker
import boto3
import uuid
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker import image_uris
from sagemaker.estimator import Estimator
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

role = <'SageMaker execution role ARN'>
sess = sagemaker.session.Session()
region = sess._region_name
bucket = sess.default_bucket()
key_prefix = "hpo-sqs"

#check HPO jobs
sm_client = boto3.client('sagemaker')

#sqs client
sqs = boto3.client('sqs')
queue_url = <'queue_url'>

HPO_LIMIT = 100


def check_hpo_jobs():
    response = sm_client.list_hyper_parameter_tuning_jobs(
    MaxResults=HPO_LIMIT,
    StatusEquals='InProgress')
    return len(list(response["HyperParameterTuningJobSummaries"]))


def create_hpo(container,train_input,validation_input,hp_range):
    print(hp_range)

    hyperparameter_ranges = {'gamma': ContinuousParameter(hp_range['gamma_lb'], hp_range['gamma_ub']),
        'alpha': ContinuousParameter(0, 2),
        'lambda': ContinuousParameter(0, 2)}
    hyperparameters={
        "num_round":"100",
        "early_stopping_rounds":"9",
        "max_depth": "5",
        "subsample": "0.9",
        "silent": "0",
        "objective": "binary:logistic",
    }
    objective_metric_name = 'validation:f1'
    xgb_churn = Estimator(
            role=role,
            image_uri=container,
            base_job_name="xgboost-churn",
            instance_count=1,
            instance_type="ml.m5.xlarge",
            hyperparameters=hyperparameters
    )

    tuner = HyperparameterTuner(xgb_churn,
                            objective_metric_name,
                            hyperparameter_ranges,
                            base_tuning_job_name = 'xgb-churn-hpo'+str(uuid.uuid4())[:3],
                            max_jobs=15,
                            max_parallel_jobs=5)
    tuner.fit({"train": train_input, "validation": validation_input}, wait=False)

def lambda_handler(event, context):

    #fist: check HPO jobs in progress
    hpo_in_progress = check_hpo_jobs()

    if hpo_in_progress>=HPO_LIMIT:
        return {
        'statusCode': 200,
        'body': json.dumps('HPO running full')
    }
    else:
        hpo_capacity = HPO_LIMIT - hpo_in_progress
        container = image_uris.retrieve("xgboost", region, "0.90-2")
        train_input = TrainingInput(f"s3://{bucket}/{key_prefix}/train/train.csv", content_type="text/csv")
        validation_input = TrainingInput(f"s3://{bucket}/{key_prefix}/validation/validation.csv", content_type="text/csv")

        while hpo_capacity> 0:
            sqs_response = sqs.receive_message(QueueUrl = queue_url)
            if 'Messages' in sqs_response.keys():
                msgs = sqs_response['Messages']
                for msg in msgs:
                    try:
                        hp_in_msg = json.loads(msg['Body'])['hyperparameter_ranges']
                        create_hpo(container,train_input,validation_input,hp_in_msg)
                        response = sqs.delete_message(QueueUrl=queue_url,ReceiptHandle=msg['ReceiptHandle'])
                        hpo_capacity = hpo_capacity-1
                        if hpo_capacity == 0:
                            break
                    except :
                        return ("error occurred for message {}".format(msg['Body']))
            else:
                return {'statusCode': 200, 'body': json.dumps('Queue is empty')}

        return {'statusCode': 200,  'body': json.dumps('Lambda completes') }


You can create Lambda function in AWS management console by copying and paste the code above. Make sure you have the following setup:
* Use Python3.7 run time. 
* Make sure you enter the right `role` ARN and `queue_url` created at setup step in the Lambda function code. 
* Add both the `sagemakersdk` layer (built from previous step) and `AWSLambda-Python37-SciPy1x` layer to your Lambda function. `AWSLambda-Python37-SciPy1x` layer is provided by AWS. You can add it to your Lambda function as follows:  click `Configuration`,  expand `Designer`,  click `Layers` and then `Add a layer`. In `Choose a layer` section, select `AWS Layers`, and select `AWSLambda-Python37-SciPy1x` from the drop down list. The correct layer setup should look as shown in the figure below.
* Set Lambda function `Timeout` to be 15 mins. This is useful when you need to create a large number of HPO jobs in one Lambda function execution. 
* Set Lambda function `Memory` to be 2048MB or more.
* Make sure the execution role of your Lambda function has permission to read from and delete messages from SQS queue and to list SageMaker hyperparameter tuning job summaries as well as to create SageMaker hyperparameter tuning jobs. 

<img src="layers.png" width = 600, height = 300>

### Set Lambda trigger

Click `Add trigger` in your Lambda function `Configuration` page. In the `Trigger configuration` page, select `EventBridge (cloudWatch Events)`, `Create a new rule` and name your `Rule name`. Make sure the `Schedule expression` option is selected, enter `rate(10 minutes)`, and click `Add`. This will trigger our Lambda function every 10 mins. 

# Prepare data for testing 

We use SageMaker session to upload data to S3. 

In [ ]:
import sagemaker
import pandas as pd
import numpy as np
role = sagemaker.get_execution_role()
sess = sagemaker.session.Session()
bucket = sess.default_bucket()
key_prefix = "hpo-sqs"

## Data

The dataset we use for testing is publicly available and was mentioned in the book [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/) by Daniel T. Larose. It is attributed by the author to the University of California Irvine Repository of Machine Learning Datasets.  Let's download and read that dataset in now:

In [ ]:
!wget http://dataminingconsultant.com/DKD2e_data_sets.zip
! apt-get install -y unzip
!unzip -o DKD2e_data_sets.zip

In [ ]:
churn = pd.read_csv('./Data sets/churn.txt')
churn = churn.drop('Phone', axis=1)
churn['Area Code'] = churn['Area Code'].astype(object)
churn = churn.drop(['Day Charge', 'Eve Charge', 'Night Charge', 'Intl Charge'], axis=1)
model_data = pd.get_dummies(churn)
model_data = pd.concat([model_data['Churn?_True.'], model_data.drop(['Churn?_False.', 'Churn?_True.'], axis=1)], axis=1)

train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])


sess.upload_string_as_file_body(body=train_data.to_csv(index=False, header=False),bucket=bucket,key=f"{key_prefix}/train/train.csv")
sess.upload_string_as_file_body(body=validation_data.to_csv(index=False, header=False),bucket=bucket,key=f"{key_prefix}/validation/validation.csv")

# Write to SQS

In [ ]:
import boto3
sqs = boto3.client('sqs')
queue_url = 'https://sqs.us-east-1.amazonaws.com/084313272408/hpo-queue'

In our message, we set the lower bound `gamma_lb` and upper bound `gamma_ub` for `gamma`, which is one of the tunable hyperparameters for demonstration purposes. You can expand the body of the message to include more fields such as other hyperparameters. 

We start by putting 150 messages in to our SQS queue.

In [ ]:
for i in range(150):
    response = sqs.send_message(
    QueueUrl=queue_url,
    DelaySeconds=1,
    MessageBody=(
        '{"hyperparameter_ranges":{"gamma_lb":0,"gamma_ub":2}}'
        )
    )

Please keep in mind our Lambda function is triggered every 10 mins. Once our Lambda function is triggered, if you check your `Hyperparameter tuning jobs` from SageMaker console, you should see HPO jobs running. 